In [14]:

class Stock:
    def __init__(self, symbol, date, open_, high, low, close, volume):
        self.symbol = symbol
        self.date = date
        self.open = open_
        self.high = high
        self.low = low
        self.close = close
        self.volume = volume

    def as_dict(self):
        return dict(symbol=self.symbol,
                    date=self.date,
                    open=self.open,
                    high=self.high,
                    low=self.low,
                    close=self.close,
                    volume=self.volume)


class Trade:
    def __init__(self, symbol, timestamp, order, price, volume, commission):
        self.symbol = symbol
        self.timestamp = timestamp
        self.order = order
        self.price = price
        self.commission = commission
        self.volume = volume

    def as_dict(self):
        return dict(
            symbol=self.symbol,
            timestamp=self.timestamp,
            order=self.order,
            price=self.price,
            volume=self.volume,
            commission=self.commission)



In [15]:
from datetime import date, datetime
from decimal import Decimal

activity = {
    "quotes": [
        Stock('TSLA', date(2018, 11, 22),
              Decimal('338.19'), Decimal('338.64'), Decimal('337.60'), Decimal('338.19'), 365_607),
        Stock('AAPL', date(2018, 11, 22),
              Decimal('176.66'), Decimal('177.25'), Decimal('176.64'), Decimal('176.78'), 3_699_184),
        Stock('MSFT', date(2018, 11, 22),
              Decimal('103.25'), Decimal('103.48'), Decimal('103.07'), Decimal('103.11'), 4_493_689)
    ],

    "trades": [
        Trade('TSLA', datetime(2018, 11, 22, 10, 5, 12), 'buy', Decimal('338.25'), 100, Decimal('9.99')),
        Trade('AAPL', datetime(2018, 11, 22, 10, 30, 5), 'sell', Decimal('177.01'), 20, Decimal('9.99'))
    ]
}

In [26]:
import json


class CustomEncoder(json.JSONEncoder):
    def default(self, arg):
        if isinstance(arg, Stock) or isinstance(arg, Trade):
            res = arg.as_dict()
            res['Object'] = arg.__class__.__name__
            return res
        elif isinstance(arg, datetime):
            return arg.isoformat()
        elif isinstance(arg, date):
            return arg.strftime('%Y-%m-%d')
        elif isinstance(arg, Decimal):
            return str(arg)
        else:
            super().default(arg)

In [27]:
res = json.dumps(activity, cls=CustomEncoder, indent=4)

In [28]:
print(res)

{
    "quotes": [
        {
            "symbol": "TSLA",
            "date": "2018-11-22",
            "open": "338.19",
            "high": "338.64",
            "low": "337.60",
            "close": "338.19",
            "volume": 365607,
            "Object": "Stock"
        },
        {
            "symbol": "AAPL",
            "date": "2018-11-22",
            "open": "176.66",
            "high": "177.25",
            "low": "176.64",
            "close": "176.78",
            "volume": 3699184,
            "Object": "Stock"
        },
        {
            "symbol": "MSFT",
            "date": "2018-11-22",
            "open": "103.25",
            "high": "103.48",
            "low": "103.07",
            "close": "103.11",
            "volume": 4493689,
            "Object": "Stock"
        }
    ],
    "trades": [
        {
            "symbol": "TSLA",
            "timestamp": "2018-11-22T10:05:12",
            "order": "buy",
            "price": "338.25",
            "vol

In [29]:
def decode_stock(d):
    s = Stock(d['symbol'],
              datetime.strptime(d['date'],
                                '%Y-%m-%d', ).date()
              , Decimal(d['open'])
              , Decimal(d['high'])
              , Decimal(d['low'])
              , Decimal(d['close'])
              , int(d['volume']))
    return s


def decode_trade(d):
    s = Trade(d['symbol'],
              datetime.strptime(d['timestamp'], '%Y-%m-%dT%H:%M:%S'),
              d['order'],
              Decimal(d['price']),
              int(d['volume']),
              Decimal(d['commission']))
    return s


def decode_financials(d):
    object_type = d.get('object', None)
    if object_type == 'Stock':
        return decode_stock(d)
    elif object_type == 'Trade':
        return decode_trade(d)
    return d


class CustomDecoder(json.JSONDecoder):
    def decode(self, arg):
        data = json.loads(arg)
        # now we have to recursively look for `Trade` and `Stock` objects
        return self.parse_financials(data)

    def parse_financials(self, obj):
        if isinstance(obj, dict):
            obj = decode_financials(obj)
            if isinstance(obj, dict):
                for key, value in obj.items():
                    obj[key] = self.parse_financials(value)
        elif isinstance(obj, list):
            for index, item in enumerate(obj):
                obj[index] = self.parse_financials(item)
        return obj

In [30]:
encoded = json.dumps(activity, cls=CustomEncoder)
decoded = json.loads(encoded, cls=CustomDecoder)

In [31]:
decoded

{'quotes': [{'symbol': 'TSLA',
   'date': '2018-11-22',
   'open': '338.19',
   'high': '338.64',
   'low': '337.60',
   'close': '338.19',
   'volume': 365607,
   'Object': 'Stock'},
  {'symbol': 'AAPL',
   'date': '2018-11-22',
   'open': '176.66',
   'high': '177.25',
   'low': '176.64',
   'close': '176.78',
   'volume': 3699184,
   'Object': 'Stock'},
  {'symbol': 'MSFT',
   'date': '2018-11-22',
   'open': '103.25',
   'high': '103.48',
   'low': '103.07',
   'close': '103.11',
   'volume': 4493689,
   'Object': 'Stock'}],
 'trades': [{'symbol': 'TSLA',
   'timestamp': '2018-11-22T10:05:12',
   'order': 'buy',
   'price': '338.25',
   'volume': 100,
   'commission': '9.99',
   'Object': 'Trade'},
  {'symbol': 'AAPL',
   'timestamp': '2018-11-22T10:30:05',
   'order': 'sell',
   'price': '177.01',
   'volume': 20,
   'commission': '9.99',
   'Object': 'Trade'}]}

In [38]:
from marshmallow import Schema, fields
from marshmallow import post_load


class StockSchema(Schema):
    symbol = fields.Str()
    date = fields.Date()
    open = fields.Decimal(as_string=True)
    high = fields.Decimal(as_string=True)
    low = fields.Decimal(as_string=True)
    close = fields.Decimal(as_string=True)
    volume = fields.Integer()

    @post_load
    def make_stock(self, data, **kwargs):
        return Stock(**data)


class TradeSchema(Schema):
    symbol = fields.Str()
    timestamp = fields.DateTime()
    order = fields.Str()
    price = fields.Decimal(as_string=True)
    commission = fields.Decimal(as_string=True)
    volume = fields.Integer()

    @post_load
    def make_trade(self, data, **kwargs):
        return Trade(**data)


class ActivitySchema(Schema):
    trades = fields.Nested(TradeSchema, many=True)
    quotes = fields.Nested(StockSchema, many=True)

In [42]:
res=ActivitySchema().dumps(activity, indent=2)
print(res)

{
  "trades": [
    {
      "symbol": "TSLA",
      "timestamp": "2018-11-22T10:05:12",
      "order": "buy",
      "price": "338.25",
      "commission": "9.99",
      "volume": 100
    },
    {
      "symbol": "AAPL",
      "timestamp": "2018-11-22T10:30:05",
      "order": "sell",
      "price": "177.01",
      "commission": "9.99",
      "volume": 20
    }
  ],
  "quotes": [
    {
      "symbol": "TSLA",
      "date": "2018-11-22",
      "open": "338.19",
      "high": "338.64",
      "low": "337.60",
      "close": "338.19",
      "volume": "365607"
    },
    {
      "symbol": "AAPL",
      "date": "2018-11-22",
      "open": "176.66",
      "high": "177.25",
      "low": "176.64",
      "close": "176.78",
      "volume": "3699184"
    },
    {
      "symbol": "MSFT",
      "date": "2018-11-22",
      "open": "103.25",
      "high": "103.48",
      "low": "103.07",
      "close": "103.11",
      "volume": "4493689"
    }
  ]
}


In [45]:
from pprint import pprint
activity_deser = ActivitySchema().loads(res)
pprint(activity_deser)

{'quotes': [{'close': Decimal('338.19'),
             'date': datetime.date(2018, 11, 22),
             'high': Decimal('338.64'),
             'low': Decimal('337.60'),
             'open': Decimal('338.19'),
             'symbol': 'TSLA',
             'volume': 365607},
            {'close': Decimal('176.78'),
             'date': datetime.date(2018, 11, 22),
             'high': Decimal('177.25'),
             'low': Decimal('176.64'),
             'open': Decimal('176.66'),
             'symbol': 'AAPL',
             'volume': 3699184},
            {'close': Decimal('103.11'),
             'date': datetime.date(2018, 11, 22),
             'high': Decimal('103.48'),
             'low': Decimal('103.07'),
             'open': Decimal('103.25'),
             'symbol': 'MSFT',
             'volume': 4493689}],
 'trades': [{'commission': Decimal('9.99'),
             'order': 'buy',
             'price': Decimal('338.25'),
             'symbol': 'TSLA',
             'timestamp': datetim